In [ ]:
] activate ../../Acceleration

In [ ]:
using Acceleration
using LinearAlgebra

n = 1000; # number of samples
d = 100;    # number of features
selected_labels = [0,1,2,3];  ## Only two labels allowed

data, labels = load_dataset("MNIST",[0,1],n,d);
f,∇f,∇ϕ_cjg,σ,L,μ  = linear_regression(data,labels);

In [ ]:
n = size(data,2)
x_ini = zeros(n)

using JuMP
using Ipopt

model = Model(with_optimizer(Ipopt.Optimizer))
set_silent(model)
@variable(model, x[1:n])
@objective(model, Min, f(x))
#@constraint(model, con, 0 .<= x)
@time optimize!(model)
x_opt = value.(x);

In [ ]:
k = 10000;
yks = zeros(n,k)
var_ξ = zeros(k)

n = length(x_ini)
x = x_ini;
y = zeros(n);
z = zeros(n);
v = zeros(n);
A0 = σ/(2*L);
A = A0;
AX = zeros(n);

yks = zeros(n,k);
μ = 0;
s = zeros(n);
p = 1
w = 0;


for i=1:k   
    λ = 1;#min((L/μ)*(p/i)^2, 1)
    a = L-μ; b = -λ*(μ*A + σ); c = -λ*(μ*A^2 + σ*A);
    α = (-b + sqrt(b^2 - 4*a*c)) / (2*a);
    A = A + α;
    x = (A-α)/A*y + α/A*v; 
    AX = AX + α*x;
    
    #q = (rand(n).-0.5)
    #ξ = 1e-1
    #η = ξ*q/norm(q);
    η = zeros(n)
    #println(norm(η))
    approx_gradient = ∇f(x)+η;
    
    z = z - α*approx_gradient;
    
    
    v_past = v;
    #v = max.(0,(z + μ*AX)*inv(1 + μ*(A-A0)));
    v = (z + μ*AX)*inv(1 + μ*(A-A0));
    #y = (A-α)*inv(A)*y + α*inv(A)*v; 
    y = x - (1/L)*approx_gradient
    
    #println( (1-λ)*(μ*(A-α) + σ)^2/α^2 *norm(v-v_past,2)^2 )
    #println(α/A)
    #println()
    println((α/A)*norm(v-v_past,2) - norm(y-x,2))
    
    #ξ = sqrt(1-λ)*(μ*(A-α) + σ)/α * norm(v-v_past,2);
    var_ξ[i] = norm(v-v_past);
    #println(ξ)
    yks[:,i] = y;
        
end

x_out = yks;
p_out = sqrt.(sum((x_out .- x_opt).^2, dims=1)');


filename = string("logs/s",p,".txt")
io = open(filename, "w")
for i=1:k
    print(io,i)
    print(io,"\t")
    print(io, p_out[i])
    print(io,"\t")
    println(io, var_ξ[i])
end
close(io)

In [ ]:
using Plots
using LinearAlgebra

p_out = sqrt.(sum((yks .- x_opt).^2, dims=1)');

plot(max.(hcat(p_out,var_ξ),1e-12),yaxis=:log,linewidth=2,xlabel="training rounds",ylabel="loss")
#plot(max.(hcat(p_out),1e-20),yaxis=:log)

In [ ]:
a = rand(n)
b = rand(n);



In [ ]:
a'*b - (0.5*norm(a,2)^2 + 0.5*norm(b,2)^2)